In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

!pip install transformers


In [ ]:
import transformers
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import tensorflow as tf
import pandas as pd
import string
import re

import matplotlib.pyplot as plt
import seaborn as sns
from transformers import BertForSequenceClassification

from transformers import TFBertModel
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/PFA/DATASET/trainset.csv",encoding='ISO-8859-1')
valid_df = pd.read_csv("/content/drive/MyDrive/PFA/DATASET/validationset.csv",encoding='ISO-8859-1')
test_df =pd.read_csv("/content/drive/MyDrive/PFA/DATASET/testset.csv",encoding='ISO-8859-1')

In [ ]:
def extract_captions(df,output):
  individual_captions = pd.DataFrame(columns=["caption","I-E","S-N","T-F","J-P"])

  for index,row in df.iterrows():
    for caption_index,caption in enumerate(eval(row['text_clean'])):
      # Define the new row to be added
      new_row = {'caption': caption.strip("\n"),"I-E":df["I-E"][index],\
                 'S-N':df["S-N"][index],'T-F':df["T-F"][index],'J-P':df["J-P"][index]}
      # Use the loc method to add the new row to the DataFrame
      individual_captions.loc[len(individual_captions)] = new_row
  individual_captions =individual_captions.drop(individual_captions[individual_captions['caption']==''].index)
  individual_captions.to_csv(output)
  return individual_captions
  

In [ ]:
train_df_indiv = extract_captions(train_df,"/content/drive/MyDrive/PFA/DATASET/indiv_trainset.csv")

In [ ]:
train_df_indiv.head()

In [ ]:
valid_df_indiv  = extract_captions(valid_df,"/content/drive/MyDrive/PFA/DATASET/indiv_validationset.csv")

In [ ]:
test_df_indiv  = extract_captions(test_df,"/content/drive/MyDrive/PFA/DATASET/indiv_testset.csv")

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained('bert-base-uncased',do_lower_case =True)

In [ ]:
def encode_caption(caption):
    tokens = tokenizer.encode_plus(
        caption,
        add_special_tokens=True,
        max_length=128,
        padding=True,truncation=True,
        return_attention_mask=True
    )
    return tokens['input_ids'], tokens['attention_mask']


def encode_individual_captions(train_df_indiv ,val_df_indiv ,test_df_indiv ):
  train_df_indiv [['input_ids', 'attention_mask']] = train_df_indiv ['caption'].apply(lambda x: pd.Series(encode_caption(x)))
  valid_df_indiv [['input_ids', 'attention_mask']] = valid_df_indiv ['caption'].apply(lambda x: pd.Series(encode_caption(x)))
  test_df_indiv [['input_ids', 'attention_mask']] = test_df_indiv ['caption'].apply(lambda x: pd.Series(encode_caption(x)))
  return train_df_indiv, valid_df_indiv, test_df_indiv


In [ ]:
train_df_indiv, valid_df_indiv, test_df_indiv = encode_individual_captions(train_df_indiv ,valid_df_indiv ,test_df_indiv )

In [ ]:
X_train = train_df_indiv[['input_ids','attention_mask']]
X_valid = valid_df_indiv[['input_ids','attention_mask']]
X_test = test_df_indiv[['input_ids','attention_mask']]

In [ ]:
y_train_ie= train_df_indiv["I-E"]
y_train_sn= train_df_indiv["S-N"]
y_train_tf= train_df_indiv["T-F"]
y_train_jp= train_df_indiv["J-P"]

y_valid_ie= valid_df_indiv["I-E"]
y_valid_sn= valid_df_indiv["S-N"]
y_valid_tf= valid_df_indiv["T-F"]
y_valid_jp= valid_df_indiv["J-P"]

y_test_ie= test_df_indiv["I-E"]
y_test_sn= test_df_indiv["S-N"]
y_test_tf= test_df_indiv["T-F"]
y_test_jp= test_df_indiv["J-P"]

In [ ]:
train_input_ids_ragged = tf.ragged.constant(X_train['input_ids'].tolist())
train_attention_mask_ragged = tf.ragged.constant(X_train['attention_mask'].tolist())

val_input_ids_ragged = tf.ragged.constant(X_valid['input_ids'].tolist())
val_attention_mask_ragged = tf.ragged.constant(X_valid['attention_mask'].tolist())

test_input_ids_ragged = tf.ragged.constant(X_test['input_ids'].tolist())
test_attention_mask_ragged = tf.ragged.constant(X_test['attention_mask'].tolist())


In [ ]:
# convert the ragged tensors to dense tensors
train_input_ids = train_input_ids_ragged.to_tensor()
train_attention_mask = train_attention_mask_ragged.to_tensor()

val_input_ids = val_input_ids_ragged.to_tensor()
val_attention_mask = val_attention_mask_ragged.to_tensor()

test_input_ids = test_input_ids_ragged.to_tensor()
test_attention_mask = test_attention_mask_ragged.to_tensor()


In [ ]:
bert_model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2
)

In [ ]:
MAX_LEN = 128

In [ ]:
def create_model(bert_model, max_len=MAX_LEN): 
  def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

  def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

  def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
  input_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32)
  attention_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32)
  output = tf.keras.layers.Dense(1, activation='sigmoid')(bert_model([input_ids, attention_mask])[1])
  model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

  
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])
  
  return model

In [ ]:
model = create_model(bert_model, MAX_LEN)

In [ ]:
model.summary()

In [ ]:
 history_bert_ie = model.fit(
    [train_input_ids, train_attention_mask], y_train_ie, 

    validation_data=([val_input_ids, val_attention_mask], y_valid_ie),

    epochs=10, batch_size=32)
 loss, accuracy, f1, precision, recall = model.evaluate([test_input_ids, test_attention_mask], y_test_ie)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test f1: { f1 }")

In [ ]:
y_pred_ie = model.predict([test_input_ids, test_atention_mask])
predictions_ie = np.round(y_pred_ie).astype(int)

In [ ]:
print('\tClassification Report for BERT FOR I-E:\n\n',classification_report(y_test_ie,predictions_ie, target_names=['0', '1']))

In [ ]:
cm = confusion_matrix(y_test_ie, predictions_ie)
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()